# Train데이터 불러오기

각 년도별 DataFrame을 리스트에 append합니다.

리스트 인덱스별로

0: 2017년도 데이터   

1: 2018년도 데이터 

2: 2019년도 데이터 

3: 202년도 데이터 

입니다

In [1]:
import pandas as pd
import numpy as np
from glob import glob

from tqdm import tqdm
import warnings

import matplotlib.pyplot as plt
import seaborn as sns

RANDOM_STATE = 42
np.seed = 42
DATA_PATH = "../data_0115/"

warnings.filterwarnings(action='ignore')
PATH_2017 = DATA_PATH + "train/KNOW_2017.csv"
PATH_2018 = DATA_PATH + "train/KNOW_2018.csv"
PATH_2019 = DATA_PATH + "train/KNOW_2019.csv"
PATH_2020 = DATA_PATH + "train/KNOW_2020.csv"

paths = [PATH_2017, PATH_2018, PATH_2019, PATH_2020]

know_train = [pd.read_csv(path) for path in paths]

In [2]:
know_train[0][['text_response', 'knowcode']] # 2017년도 샘플

,text_response,knowcode
0,자동차도장기능사 실무교육 생산 설비 자동화 없다,825101
1,건축전기설비기술사 건설 수주 드라이버 가위,140204
2,건축전기설비기술사 신축 건설 경기 건축 설계 오토캐드 파이프 보일러,140204
3,지적기사 측량 정보통신기술 교육 무량 감소 자동화 토탈 스테이션,140601
4,건축전기설비기술사 부동산 거품 건축 경기도 덩달아 불황 캐드 엑셀 건축 도면,140204
...,...,...
9481,편집기술에 원리와 이해능력 출판 업무 특성 기획 업무 한글 편집 프로그램 포토샵 일...,411301
9482,도면 작성 교육 전자 기초 이해 항공우주 분야 국가 경쟁력 위해 육성 전문 특수 정...,151105
9483,건설 경기 볼트 리벳 없다,701101
9484,공무원 인원 크게 변동 컴퓨터,25402


# 전처리 

`text_response`를 거대한 column으로 만듭니다.  
매우매우 sparse

In [3]:
stop_words = '''
없다 증가 없음 감소 때문 수요 대한 모름 도구 사람 산업 관련 유지 업무 일자리 관심
계속 인력 발달 발전 필요 직업 분야 대체 환경 추세 사회 확대
'''.split()

In [4]:
years = ['2017', '2018', '2019', '2020']

In [6]:
# Preprocess Train Data
from os. path import isfile

keywords_list_dic = {}
for i in range(4):
    print(years[i])
    # if file exists, Load it!
    data_path = f'../data_keywords/v1/train/keywords_{years[i]}.csv'
    know_code_list = know_train[i]['knowcode'].unique().tolist()
    if isfile(data_path):
        new_df = pd.read_csv(data_path)
        new_df['knowcode'] = know_train[i]['knowcode']
        know_train[i] = new_df
        print(f"read from file {data_path}")
        continue

    # Get Keywords
    know_code_set = set()
    for know_code in know_code_list:
        df = know_train[i][know_train[i]['knowcode'] == know_code]
        big_txt = df.groupby('knowcode')['text_response'].apply(lambda x: ' '.join(x)).iloc[0]

        word_cnt_dict = {}
        for w in big_txt.split():
            if w in stop_words:
                continue

            if w not in word_cnt_dict.keys():
                word_cnt_dict[w] = 1
            else:
                word_cnt_dict[w] += 1

        l_size = 10
        l = sorted(word_cnt_dict.items(), key=lambda x:x[1], reverse=True)[:l_size]
        know_code_set.update([x[0] for x in l])

    keywords_list = list(know_code_set)
    keywords_list_dic[i] = keywords_list
    print(f"keywords_list length: {len(keywords_list)}")

    # Builds DataFrame based on the keywords
    def word_count(l):
        word_cnt_dict = {}
        for w in l:
            if w not in keywords_list:
                continue
            if w not in word_cnt_dict.keys():
                word_cnt_dict[w] = 1
            else:
                word_cnt_dict[w] += 1
        
        return sorted(word_cnt_dict.items(), key=lambda x:x[1], reverse=True)
    sr = know_train[i]['text_response'].apply(lambda x: word_count(x.split()))

    new_df = pd.DataFrame(columns=keywords_list)
    row_list = []

    for row in tqdm(sr):
        temp_dict = {}
        for k, v in row:
            temp_dict[k] = v
        new_df = new_df.append(temp_dict, ignore_index=True)

    new_df.index = know_train[i].index
    new_df.fillna(0, inplace=True)
    
    new_df.to_csv(data_path, index=False)
    new_df['knowcode'] = know_train[i]['knowcode']
    print(f"new_df.shape = {new_df.shape}")
    know_train[i] = new_df

2017
keywords_list length: 1952


100%|██████████| 9486/9486 [04:14<00:00, 37.24it/s] 


new_df.shape = (9486, 1953)
2018
keywords_list length: 2437


100%|██████████| 9072/9072 [04:38<00:00, 32.58it/s] 


new_df.shape = (9072, 2438)
2019
keywords_list length: 1853


100%|██████████| 8555/8555 [02:58<00:00, 47.80it/s] 


new_df.shape = (8555, 1854)
2020
keywords_list length: 1791


100%|██████████| 8122/8122 [02:41<00:00, 50.36it/s] 


new_df.shape = (8122, 1792)


# X, y 구분 및 모델 학습

이번 대회에서 맞춰야 할 값은 knowcode입니다.

ID와 knowcode를 제외한 나머지 feature를 X, knowcode를 정답 y로 두어 모델을 학습하였습니다.

베이스라인에서는 의사결정나무와 랜덤포레스트를 선정하였습니다

df.drop(['idx','knowcode'], axis=1)

In [7]:
train_data = {}
for year, df in zip(years, know_train):
    train_data[year] = {'X': df.drop('knowcode', axis=1),
                        'y': df['knowcode']} 

In [8]:
train_data['2017']

{'X':        티칭   통관   급증   행정   압착  모니터  조종사   발성  우편물   촬영  ...  연구기법   공장   통계  \
 0     0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0  0.0  0.0   
 1     0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0  0.0  0.0   
 2     0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0  0.0  0.0   
 3     0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0  0.0  0.0   
 4     0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0  0.0  0.0   
 ...   ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   ...  ...  ...   
 9481  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0  0.0  0.0   
 9482  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0  0.0  0.0   
 9483  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0  0.0  0.0   
 9484  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0  0.0  0.0   
 9485  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0  0.0  0.0   
 
       어려움  기기사용법  치과의사   부서  컨베이

# Naive Bayesian

In [10]:
from sklearn.naive_bayes import GaussianNB

gnb_models = {}

for year in tqdm(years):
    model = GaussianNB()
    model.fit(train_data[year]['X'].iloc[:, :], train_data[year]['y'])
    gnb_models[year] = model

100%|██████████| 4/4 [00:01<00:00,  3.83it/s]


## Testset 불러오기

마찬가지로 년도별로 DataFrame으로 불러온 후 리스트에 할당합니다.


In [11]:
TEST_PATH_2017 = DATA_PATH + "test/KNOW_2017_test.csv"
TEST_PATH_2018 = DATA_PATH + "test/KNOW_2018_test.csv"
TEST_PATH_2019 = DATA_PATH + "test/KNOW_2019_test.csv"
TEST_PATH_2020 = DATA_PATH + "test/KNOW_2020_test.csv"

TEST_PATHs = [TEST_PATH_2017, TEST_PATH_2018, TEST_PATH_2019, TEST_PATH_2020]

In [13]:
know_test = [pd.read_csv(path) for path in TEST_PATHs]
know_test[0] # 2017년도 test 샘플

,idx,aq1_1,aq1_2,aq2_1,aq2_2,aq3_1,aq3_2,aq4_1,aq4_2,aq5_1,...,bq41_3,text_response,sim_job,bef_job,able_job,major,bq6_mark,bq12_2_mark,bq12_3_mark,bq12_4_mark
0,0,3,4,2,2,3,3,1,0,3,...,2300,대체 업무 컴퓨터 없다,없다,없다,없다,비서학,1,0,0,0
1,1,5,5,3,5,5,5,5,5,4,...,2500,품질관리기사 실무교육 제조업 생각 때문 제품 검사 시스템 엑셀,없다,없다,없다,농화학,0,0,0,0
2,2,5,5,5,4,5,4,1,0,1,...,4000,기획력 현장경험 미디어 증가 오디션 편집 프로그램 홀로 방송 진행 장비 1인 미디어...,없다,없다,"광선, 홍보 담당자",신문방송,0,0,0,0
3,3,4,5,5,6,4,6,3,4,4,...,3000,선호 직업 컴퓨터 없다,없다,없다,없다,화학,1,0,0,0
4,4,5,6,4,5,4,5,1,0,1,...,2000,잡지 정보 뉴스 고갈 때문 유지 뉴스 원고 없다,편집기자,사진작가,리포터,광고홍보,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9481,9481,3,4,4,5,4,5,5,6,4,...,0,굴삭기 운전 변동 와이어 굴삭기 없다,없다,없다,없다,0,0,0,0,0
9482,9482,4,5,4,6,5,6,5,6,5,...,2400,도면의 이해 기본서류작성 PPT EXCEL 선진 사의 기술 중국 자본 저가 공세 없다,없다,없다,없다,요업과,0,0,0,0
9483,9483,3,2,1,0,2,1,3,3,1,...,0,건설기계 면허 업종 기피,없다,자영업(과일과게),없다,인문계,1,0,0,0
9484,9484,4,5,3,4,3,4,1,0,1,...,4000,CCIE 정보처리기사 직무교육 투자 확대 없다,없다,없다,없다,컴퓨터공학,0,0,0,0


In [14]:
# Preprocess Test Data

for i in range(4):
    print(years[i])
    # if file exists, Load it!
    data_path = f'../data_keywords/v1/test/keywords_{years[i]}_test.csv'
    if isfile(data_path):
        new_df = pd.read_csv(data_path)
        new_df['knowcode'] = know_test[i]['knowcode']
        know_test[i] = new_df
        continue

    # Get Keywords
    keywords_list = keywords_list_dic[i] 
    print(f"keywords_list length: {len(keywords_list)}")

    # Builds DataFrame based on the keywords
    def word_count(l):
        word_cnt_dict = {}
        for w in l:
            if w not in keywords_list:
                continue
            if w not in word_cnt_dict.keys():
                word_cnt_dict[w] = 1
            else:
                word_cnt_dict[w] += 1
        return sorted(word_cnt_dict.items(), key=lambda x:x[1], reverse=True)
    sr = know_test[i]['text_response'].apply(lambda x: word_count(x.split()))

    new_df = pd.DataFrame(columns=keywords_list)
    row_list = []

    for row in tqdm(sr):
        temp_dict = {}
        for k, v in row:
            temp_dict[k] = v
        new_df = new_df.append(temp_dict, ignore_index=True)

    new_df.index = know_test[i].index
    new_df.fillna(0, inplace=True)
    
    new_df.to_csv(data_path, index=False)
    print(f"new_df.shape = {new_df.shape}")
    know_test[i] = new_df

2017
keywords_list length: 1952


100%|██████████| 9486/9486 [04:07<00:00, 38.32it/s] 


new_df.shape = (9486, 1952)
2018
keywords_list length: 2437


100%|██████████| 9069/9069 [04:56<00:00, 30.62it/s] 


new_df.shape = (9069, 2437)
2019
keywords_list length: 1853


100%|██████████| 8554/8554 [03:09<00:00, 45.21it/s] 


new_df.shape = (8554, 1853)
2020
keywords_list length: 1791


100%|██████████| 8122/8122 [02:49<00:00, 47.80it/s] 


new_df.shape = (8122, 1791)


## 테스트셋 추출 및 학습

ID 값을 제외한 나머지 데이터를 이용하여 모델에 넣어 추론합니다.

In [15]:
test_data = {}
for year, df in zip(years, know_test):
    print(year)
    train_columns = train_data[year]['X'].columns
    # print(train_columns)
    test_data[year] =  {'X': df[train_columns]}

2017
2018
2019
2020


In [16]:
for year in years:
    print(year)
    print(f"train: {train_data[year]['X'].shape} test: {test_data[year]['X'].shape}")

2017
train: (9486, 1952) test: (9486, 1952)
2018
train: (9072, 2437) test: (9069, 2437)
2019
train: (8555, 1853) test: (8554, 1853)
2020
train: (8122, 1791) test: (8122, 1791)


### NB로 예측

In [17]:
gnb_predicts = [] 

for year in tqdm(years):
    pred = gnb_models[year].predict(test_data[year]['X'])
    gnb_predicts.extend(pred)

100%|██████████| 4/4 [02:29<00:00, 37.48s/it]


# 제출

In [18]:
submission = pd.read_csv('../data_0103/sample_submission.csv') # sample submission 불러오기

In [19]:
submission['knowcode'] = gnb_predicts

submission.to_csv('../submission_files/nb_onlytext_with_data0115.csv', index=False)